# Gold Order Priorities Table load

In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import * 

StatementMeta(, 1c87a4e7-6aac-4b45-a69b-cb3980039db5, 4, Finished, Available, Finished)

In [3]:
# Here we will create table and implement incremental logic using pysaprk

StatementMeta(, 1c87a4e7-6aac-4b45-a69b-cb3980039db5, 5, Finished, Available, Finished)

In [4]:
DeltaTable.createIfNotExists(spark)\
          .tableName("Gold_OrderPriority")\
          .addColumn("OrderPriority_ID",LongType())\
          .addColumn("Order_Priority",StringType())\
          .addColumn("Created_TS",TimestampType())\
          .addColumn("Modified_TS",TimestampType())\
          .execute()

StatementMeta(, 1c87a4e7-6aac-4b45-a69b-cb3980039db5, 6, Finished, Available, Finished)

In [5]:
#assigne the created table to df
df=spark.read.table("ecom_proj_LH.Gold_OrderPriority")

StatementMeta(, 1c87a4e7-6aac-4b45-a69b-cb3980039db5, 7, Finished, Available, Finished)

In [6]:
Max_Date=df.selectExpr("coalesce(max(Modified_TS),'1900-01-01')").first()[0]

StatementMeta(, 1c87a4e7-6aac-4b45-a69b-cb3980039db5, 8, Finished, Available, Finished)

In [7]:
Max_Date

StatementMeta(, 1c87a4e7-6aac-4b45-a69b-cb3980039db5, 9, Finished, Available, Finished)

'1900-01-01'

In [8]:
# Now we need to take required columns from bronze table into order priority table

StatementMeta(, 1c87a4e7-6aac-4b45-a69b-cb3980039db5, 10, Finished, Available, Finished)

In [9]:
df_bronze = spark.read.table("ecom_proj_LH.bronze_sales")

StatementMeta(, 1c87a4e7-6aac-4b45-a69b-cb3980039db5, 11, Finished, Available, Finished)

In [10]:
df_brinze_mod=df_bronze.select("Order_Priority").where(col("Modified_TS")>Max_Date).drop_duplicates()

StatementMeta(, 1c87a4e7-6aac-4b45-a69b-cb3980039db5, 12, Finished, Available, Finished)

In [12]:
# Now we'll add incremental column, we'll use montonically increaing function

StatementMeta(, 1c87a4e7-6aac-4b45-a69b-cb3980039db5, 14, Finished, Available, Finished)

In [14]:
Max_ID = df.selectExpr("coalesce(max(OrderPriority_ID),0)").first()[0]

df_final=df_brinze_mod.withColumn("OrderPriority_ID",Max_ID+monotonically_increasing_id()+1)

StatementMeta(, 1c87a4e7-6aac-4b45-a69b-cb3980039db5, 16, Finished, Available, Finished)

In [15]:
df_final.show()

StatementMeta(, 1c87a4e7-6aac-4b45-a69b-cb3980039db5, 17, Finished, Available, Finished)

+--------------+----------------+
|Order_Priority|OrderPriority_ID|
+--------------+----------------+
|          High|               1|
|           Low|               2|
|        Medium|               3|
|      Critical|               4|
+--------------+----------------+



In [19]:
#converting gold priority table into spark data frame
df_gold_delta=DeltaTable.forPath(spark,"Tables/gold_orderpriority")
df_bronze_table = df_final

StatementMeta(, 1c87a4e7-6aac-4b45-a69b-cb3980039db5, 21, Finished, Available, Finished)

In [22]:
df_gold_delta.alias("gold")\
             .merge(df_bronze_table.alias("bronze"),"gold.Order_Priority==Bronze.Order_Priority")\
             .whenMatchedUpdate(set={"gold.Modified_TS":current_timestamp()})\
             .whenNotMatchedInsert(values={
                "gold.OrderPriority_ID":"Bronze.OrderPriority_ID",
                "gold.Order_Priority":"Bronze.Order_Priority",
                "gold.Created_TS":current_timestamp(),
                "gold.Modified_TS":current_timestamp()
             }).execute()

StatementMeta(, 1c87a4e7-6aac-4b45-a69b-cb3980039db5, 24, Finished, Available, Finished)

StatementMeta(, 1c87a4e7-6aac-4b45-a69b-cb3980039db5, 25, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 4 fields>